In [ ]:
%load_ext autoreload
%autoreload 2
from benchmark.preprocess import *
from benchmark.constants import *
from benchmark.db_import import *
from benchmark.ground_truth import *
from benchmark.construct_threatrace_graph import *
from benchmark.threatrace import *
import logging 
from torch_geometric import seed_everything
import os
import configparser
from patricks_modul import PatricksLog
logging.basicConfig(
    format='%(asctime)s - %(levelname)s - %(message)s',
    level=logging.INFO,
    filename='app.log',
    #filemode='w'
    filemode='a',
)
logging.info('This will get logged to a file')



patricks_log = PatricksLog('sagegcngat_0')

In [ ]:
project_dir = os.getcwd()
print("Current project directory: ", project_dir)
patricks_log.write_kv_to_file('Current project directory', project_dir)

In [ ]:
# Read config file
config_dir = "../configs"
config_file_record_03 = config_dir + '/eng3/cadets/cadets_03_record.ini'
config_rec3 = configparser.ConfigParser()
config_rec3.read(config_file_record_03)

In [ ]:
# read tar file and extract to a folder + preprocess
process_darpa_data(config_rec3)


In [ ]:
#setup and import data to mariaDB 
import_nodes_and_edges(config_rec3)

In [ ]:
print(config_rec3['threatrace'])

In [ ]:
own_timestamp_percent=20
patricks_log.write_kv_to_file('Test/Train split', own_timestamp_percent)
# create train and test data
mypygdata = MyPyGDataset(config_rec3, own_timestamp_percent, extension="20percentTrain") # 4min
data_train = mypygdata[0][0]
data_test = mypygdata[0][1]
print('Data_train:')
print(data_train)
print('Data_train_x: ')
print(data_train['x'][:1])
print('Data_train_edge_index: ')
print(data_train['edge_index'][:2])
print('Verteilung Train', np.bincount(data_train['y']))
print('Data_test: ')
print(data_test)
print('Verteilung Test', np.bincount(data_test['y']))
patricks_log.write_div_to_file()
patricks_log.write_kv_to_file('Data_train size of x / features', str(list(data_train.x.size())))
patricks_log.write_kv_to_file('Data_train size of edge_index', str(list(data_train.edge_index.size())))
patricks_log.write_kv_to_file('Data_train size of y', str(list(data_train.y.size())))
patricks_log.write_kv_to_file('Data_train distribution of y', str(np.bincount(data_train['y'])))
patricks_log.write_kv_to_file('Data_train size of train_mask / test_mask', str(list(data_train.train_mask.size())) + str(list(data_train.test_mask.size())))
patricks_log.write_to_file('')
patricks_log.write_kv_to_file('Data_test size of x / features', str(list(data_test.x.size())))
patricks_log.write_kv_to_file('Data_test size of edge_index', str(list(data_test.edge_index.size())))
patricks_log.write_kv_to_file('Data_test size of y', str(list(data_test.y.size())))
patricks_log.write_kv_to_file('Data_test distribution of y', str(np.bincount(data_test['y'])))
patricks_log.write_kv_to_file('Data_test size of train_mask / test_mask', str(list(data_test.train_mask.size())) + str(list(data_test.test_mask.size())))
patricks_log.save_time_since_start()
patricks_log.write_div_to_file()

In [ ]:
# create ground truth
gt_filename = 'gt.txt'
gt_load = True

if gt_load and os.path.exists(gt_filename):
    with open(gt_filename, 'r') as file:
        gt = [int(x) for x in file.readline().split()]
else:
    gt_file1 = config_rec3['Files']['ground_truth'] # -> special case for record3, because here 2 attacks are given 
    gt_file2 = config_rec3['Files']['ground_truth1'] # -> special case for record3, because here 2 attacks are given 
    gts = [gt_file1, gt_file2]
    gt = get_atk_nodes_for_evaluation(config_rec3, multi=True, gts=gts) # -> list with id´s [526, 524824, 537, 262769, ...]

    with open(gt_filename, 'w') as file:
        file.write(' '.join(map(str, gt)))

print(len(gt))

In [ ]:
# create threatrace GNN anomaly detection Pipeline 
ttp = ThreaTracePipeline(config_rec3, data_train, data_test, patricks_log) 
patricks_log.save_time_since_start()

In [ ]:
ttp.delete_old_models()
patricks_log.write_to_file('### PRETRAINING ### - wird nichtgesaved')
ttp.pretraining()
patricks_log.save_time_since_start()

In [ ]:
patricks_log.write_to_file('\n### MULTI MODEL TRAINING ###')
ttp.multi_model_training()
patricks_log.save_time_since_start()

In [ ]:
print("##############################################################################################################")
print("own evaluation")
print("##############################################################################################################")
patricks_log.write_to_file('\n### OWN EVALUATION ### - Jedes Model mit dem Test Datensatz testen')
ttp.reinit_test_data() 
ttp.test_model_performance(gt) 
patricks_log.save_time_since_start()

In [ ]:
print("##############################################################################################################")
print("0 HOP")
patricks_log.write_to_file('\n### 0 HOP ###')
hits = ttp.evaluation_single_hop(gt)
patricks_log.save_time_since_start()

In [ ]:
print("##############################################################################################################")
print("1 HOP")
patricks_log.write_to_file('\n### 1 HOP ###')
hits = ttp.evaluation(gt, 1)
patricks_log.save_time_since_start()

In [ ]:
print("##############################################################################################################")
print("2 HOP")
patricks_log.write_to_file('\n### 2 HOP ###')
hits = ttp.evaluation(gt, 2)
patricks_log.save_time_since_start()

In [ ]:
print("##############################################################################################################")
print("Detection insights")
x = ttp.get_detection_insights()
patricks_log.save_time_since_start()